In [0]:
!git clone https://github.com/exowanderer/vaelstmpredictor

In [0]:
!pip install paramiko

In [0]:
import os
os.chdir('/content/vaelstmpredictor')

In [0]:
!git checkout sql_based_ga

In [0]:
!git branch

In [0]:
!git pull

In [0]:
pwd

In [0]:
from TrainChromosome_Conv1D import *

In [0]:
hostname = 'colab'

In [0]:
class blankClass(object):
	def __init__(self):
		pass

In [0]:
clargs = blankClass()

In [0]:
while True:
  try:
    chromosome = requests.get(url="http://LAUDeepGenerativeGenetics.pythonanywhere.com/GetUnTrainedChrom")
  except:
    chromosome = None
  if(chromosome != None and chromosome.text == "0"):
    info_message("No more Chromosomes to train")
    sleep(30)
  elif(chromosome != None):
    try:
      params = chromosome.json()		
    except:		
      continue

    clargs.batch_size = params["batch_size"]
    clargs.chromosomeID = params["chromosomeID"]
    clargs.cross_prob = params["cross_prob"]
    clargs.dnn_kl_weight = params["dnn_kl_weight"]
    clargs.dnn_log_var_prior = params["dnn_log_var_prior"]
    clargs.dnn_weight = params["dnn_weight"]
    clargs.fitness = params["fitness"]
    clargs.generationID = params["generationID"]
    clargs.isTrained = params["isTrained"]
    clargs.kl_anneal = params["kl_anneal"]
    clargs.log_dir = params["log_dir"]
    clargs.do_log = True
    clargs.do_ckpt = False
    clargs.verbose = True
    clargs.save_model = False
    #clargs.model_dir = params["model_dir"]
    clargs.model_dir = "data/models"
    clargs.mutate_prob = params["mutate_prob"]
    clargs.num_dnn_layers = params["num_dnn_layers"]
    clargs.num_epochs = params["num_epochs"]
    clargs.num_generations = params["num_generations"]
    clargs.num_vae_layers = params["num_vae_layers"]
    clargs.optimizer = params["optimizer"]
    clargs.patience = params["patience"]
    clargs.population_size = params["population_size"]
    clargs.prediction_log_var_prior = params["prediction_log_var_prior"]
    clargs.predictor_type = params["predictor_type"]
    clargs.run_name = params["run_name"]
    clargs.size_dnn_hidden = params["size_dnn_hidden"]
    clargs.size_vae_hidden = params["size_vae_hidden"]
    clargs.vae_latent_dim = params["size_vae_latent"]
    clargs.table_dir = params["table_dir"]
    clargs.train_file = params["train_file"]
    clargs.vae_kl_weight = params["vae_kl_weight"]
    clargs.vae_weight = params["vae_weight"]
    clargs.w_kl_anneal = params["w_kl_anneal"]
    clargs.num_conv_layers = params["num_conv_layers"]
    clargs.size_kernel = params["size_kernel"]
    clargs.size_pool = params["size_pool"]
    clargs.size_filter = params["size_filter"]

    clargs.hostname = hostname
    clargs.time_stamp = int(time())

    vae_hidden_dims = [clargs.size_vae_hidden]*clargs.num_vae_layers
    dnn_hidden_dims = [clargs.size_dnn_hidden]*clargs.num_dnn_layers
    size_kernel = np.array(json.loads(clargs.size_kernel))
    size_pool = np.array(json.loads(clargs.size_pool))
    size_filter = np.array(json.loads(clargs.size_filter))

    data_instance = MNISTData(batch_size = clargs.batch_size)

    n_train, n_features = data_instance.data_train.shape
    n_test, n_features = data_instance.data_valid.shape

    clargs.original_dim = n_features
    clargs.n_labels = len(np.unique(data_instance.train_labels))

    chrom_params = {}
    chrom_params['data_instance'] = data_instance
    chrom_params['verbose'] = clargs.verbose
    chrom_params['save_model'] = clargs.save_model
    chrom_params['vae_hidden_dims'] = vae_hidden_dims
    chrom_params['dnn_hidden_dims'] = dnn_hidden_dims
    chrom_params['vae_latent_dim'] = clargs.vae_latent_dim
    chrom_params['generationID'] = clargs.generationID
    chrom_params['chromosomeID'] = clargs.chromosomeID
    chrom_params['vae_weight'] = clargs.vae_weight
    chrom_params['vae_kl_weight'] = clargs.vae_kl_weight
    chrom_params['dnn_weight'] = clargs.dnn_weight
    chrom_params['dnn_kl_weight'] = clargs.dnn_kl_weight
    chrom_params['num_conv_layers'] = clargs.num_conv_layers
    chrom_params['size_kernel'] = size_kernel
    chrom_params['size_pool'] = size_pool
    chrom_params['size_filter'] = size_filter
    chrom_params['clargs'] = clargs

    info_message('\n\nParams for this VAE_NN:')
    for key,val in clargs.__dict__.items():
      print('{:20}{}'.format(key,val))

    info_message("Training Chromosome "+str(clargs.chromosomeID)+" Generation "+str(clargs.generationID))
    try:
      chromosome = Chromosome(**chrom_params)
      chromosome.verbose = True
      chromosome.train(verbose=True)
      chromosome.model.summary()
      chromosome.train(verbose=True)
      K.clear_session()
    except Exception as e:
      warning_message("Error has occured while training")
      print(e)
      chromosome = None
      continue

    info_message('\n')
    print('Result: ', end=" ")
    print('GenerationID: {}'.format(chromosome.generationID), end=" ")
    print('ChromosomeID: {}'.format(chromosome.chromosomeID), end=" ")
    print('Fitness: {}\n'.format(chromosome.fitness))

    params["isTrained"] = 2
    params["fitness"] = chromosome.fitness
    params["val_vae_reconstruction_loss"] = chromosome.best_loss['val_vae_reconstruction_loss']
    params["val_vae_latent_args_loss"] = chromosome.best_loss['val_vae_latent_args_loss']
    params["val_dnn_latent_layer_loss"] = chromosome.best_loss['val_dnn_latent_layer_loss']
    params["val_dnn_latent_mod_loss"] = chromosome.best_loss['val_dnn_latent_mod_loss']
    params["hostname"] = clargs.hostname
    params["time_stamp"] = clargs.time_stamp

    sent = False
    while(not sent):
      try:
        resp = requests.get(url="http://LAUDeepGenerativeGenetics.pythonanywhere.com/AddChrom", params=params)
        info_message("Response: "+resp.text)
        sent = (resp.text == "1")
      except:
        sent = False
